In [7]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import numpy as np

In [9]:
load_dotenv()

token = os.getenv('TOKEN')

import sys
#print paths
sys.path.append('..')
print(sys.path)
df = pd.read_csv("dataset/dataset.csv", delimiter=';')

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/sersasj/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '../', '..']


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/dataset.csv'

In [4]:
df.shape

(378, 49)

In [8]:
df_filtrado = df[df['Archived'] == 'No']

print(df_filtrado.shape  )



In [ ]:
df_filtrado = df_filtrado[df_filtrado['Application Purpose'] == 'Production ready']

In [9]:
df_filtrado.shape

(284, 49)

In [12]:
total_closed_issues_list = []

url = "https://api.github.com/graphql"
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": "2022-11-28"
}

for index, row in df_filtrado.iterrows():
    urlRepo = row['URL']

    parts = urlRepo.split('https://github.com/')[1].split('/')
    owner = parts[0]
    repo = parts[1]

    query = f"""
    {{
        repository(owner: "{owner}", name: "{repo}") {{
            issues(states: CLOSED) {{
                totalCount
            }}
        }}
    }}
    """

    payload = {
        "query": query
    }

    try:
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            data = response.json()

            if data['data']['repository'] is not None:
                total_closed_issues = data['data']['repository']['issues']['totalCount']
                total_closed_issues_list.append({
                    'index': index,
                    'quantidade': total_closed_issues
                })
            else:
                error_message = data['errors'][0]['message']
                total_closed_issues_list.append({
                    'index': index,
                    'erro': error_message
                })
        else:
            total_closed_issues_list.append({
                'index': index,
                'erro': f"Erro HTTP {response.status_code}: {response.text}"
            })

    except Exception as e:
        total_closed_issues_list.append({
            'index': index,
            'erro': str(e)
        })

print(total_closed_issues_list)

[{'index': 1, 'quantidade': 0}, {'index': 2, 'quantidade': 0}, {'index': 3, 'quantidade': 3737}, {'index': 4, 'quantidade': 2454}, {'index': 5, 'quantidade': 87}, {'index': 6, 'quantidade': 0}, {'index': 7, 'quantidade': 0}, {'index': 8, 'erro': "Could not resolve to a Repository with the name 'incognitochain/incognito-chain'."}, {'index': 9, 'quantidade': 6642}, {'index': 10, 'quantidade': 1389}, {'index': 11, 'quantidade': 1908}, {'index': 12, 'quantidade': 11079}, {'index': 13, 'quantidade': 0}, {'index': 14, 'quantidade': 834}, {'index': 15, 'quantidade': 0}, {'index': 16, 'quantidade': 2503}, {'index': 17, 'quantidade': 2523}, {'index': 18, 'quantidade': 3801}, {'index': 19, 'quantidade': 485}, {'index': 20, 'quantidade': 3243}, {'index': 21, 'quantidade': 2426}, {'index': 22, 'quantidade': 1425}, {'index': 24, 'quantidade': 4850}, {'index': 25, 'quantidade': 2347}, {'index': 26, 'quantidade': 79}, {'index': 27, 'quantidade': 1020}, {'index': 28, 'quantidade': 3307}, {'index': 29,

In [24]:
valid_indexes = [item['index'] for item in total_closed_issues_list if 'erro' not in item]

df_filtrado = df_filtrado[df_filtrado.index.isin(valid_indexes)]

In [27]:
quantidades = [item['quantidade'] for item in total_closed_issues_list if 'quantidade' in item]

media_quantidades_issues = np.mean(quantidades)
mediana_quantidades_issues = np.median(quantidades)
dp_quantidades_issues = np.std(quantidades)

print(f"Média das quantidades: {media_quantidades_issues}")
print(f"Mediana das quantidades: {mediana_quantidades_issues}")
print(f"Desvio padrão das quantidades: {dp_quantidades_issues}")

Média das quantidades: 428.7864768683274
Mediana das quantidades: 30.0
Desvio padrão das quantidades: 1083.8161706555316


In [29]:
indexes_maior_que_media = [item['index'] for item in total_closed_issues_list if 'quantidade' in item and item['quantidade'] > media_quantidades_issues]

df_filtrado = df_filtrado[df_filtrado.index.isin(indexes_maior_que_media)]

In [31]:
df_filtrado.shape

(56, 49)

In [33]:
mediana_community_size = df_filtrado['CommunitySize'].median()
media_community_size = df_filtrado['CommunitySize'].mean()
dp_community_size = df_filtrado['CommunitySize'].std()

print(f"Média da comunidade: {mediana_community_size}")
print(f"Mediana da comunidade: {media_community_size}")
print(f"Desvio padrão da comunidade: {dp_community_size}")

Média da comunidade: 155.5
Mediana da comunidade: 349.67857142857144
Desvio padrão da comunidade: 728.4256025439786


In [34]:
df_filtrado = df_filtrado[df_filtrado['CommunitySize'] > media_community_size]

In [35]:
df_filtrado.shape

(11, 49)

In [36]:
df_filtrado.to_csv('dataset_filtrado.csv', index=False,  sep=';')